### Clasificación. Regresión logística.

Otro tipo de problema típico a resolver con modelos estadísticos es el de clasificación. De la unidad 8 recordarás que el problema aquí consiste en estimar la categoría más probable a la que pertenece un elemento en función de una serie de características observadas, que utilizamos como variables explicativas del modelo.

Existen varias técnicas de clasificación estadística. Pero ahora vamos a centrarnos en la regresión logística. 

Statsmodels incluye métodos y utilizades específicas para regresión logística. La interfaz principal es `sm.Logit()`. Como vas a ver, su uso no puede ser más sencillo.

Vamos a reutilizar el ejemplo de la unidad 8 para intentar determinar el sexo de una persona en función de varias medidas antropométricas.

In [ ]:
import sys
import os.path

# Tan solo modifica la cadena de texto entre comillas
# de DIR_FICHEROS_CURSO
# Sustituye por la ruta completa hasta el directorio base
# donde descargues el material del campus
DIR_FICHEROS_CURSO = "."
DIR_U09 = os.path.join(DIR_FICHEROS_CURSO, "09PythonParaAnalisisDatos")
DIR_U09_SRC = os.path.join(DIR_U09, "U09_src")

sys.path.append(DIR_U09_SRC)

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

body = pd.read_csv("../../Datasets/body_measures_subset.csv", delimiter=";")

# Los datos contienen varias medidas antropométricas
# `Gender` es una variable categórica
#  - 0 : mujer
#  - 1 : hombre
body.head()

,Gender,Ankle_min_girth,Wrist_min_girth,Height,Weight,Chest_depth,Biacromial_diameter
0,1,23.5,16.5,174.0,65.6,17.7,42.9
1,1,24.5,17.0,175.3,71.8,16.9,43.7
2,1,21.9,16.9,193.5,80.7,20.9,40.1
3,1,23.0,16.6,186.5,72.6,18.4,44.3
4,1,24.4,18.0,187.2,78.8,21.5,42.5


In [ ]:
# Variable respuesta
y = body['Gender']
# Las variables predictoras son todas las demás
X = body.iloc[:,1:]
# Añadimos el término constante
X = sm.add_constant(X, prepend=False)

# Construimos el modelo logístico
model = smf.Logit(y, X)
# Y ajustamos
mfitted = model.fit()

Optimization terminated successfully.
         Current function value: 0.154336
         Iterations 9


In [ ]:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [ ]:
# Examinamos los resultados del ajuste
print(mfitted.summary())

                           Logit Regression Results                           
Dep. Variable:                 Gender   No. Observations:                  507
Model:                          Logit   Df Residuals:                      500
Method:                           MLE   Df Model:                            6
Date:                Tue, 16 Jul 2019   Pseudo R-squ.:                  0.7772
Time:                        19:20:41   Log-Likelihood:                -78.248
converged:                       True   LL-Null:                       -351.26
                                        LLR p-value:                1.018e-114
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Ankle_min_girth        -0.4816      0.235     -2.050      0.040      -0.942      -0.021
Wrist_min_girth         2.6984      0.444      6.075      0.000       1.828       3.569
Height          

Podemos ver la matriz de confusión del ajuste con el método `pred_table()`. Las filas corresponden a las observaciones (0 y 1) de la variable respuesta, mientras que las columnas corresponden a los valores estimados por el modelo ajustado sobre los datos de entrenamiento. La diagonal recoge los casos correctos.

In [ ]:
print(mfitted.pred_table())

[[246.  14.]
 [ 16. 231.]]


Pero naturalmente, lo importante son los resultados sobre datos distintos de los usados para entrenar. Vamos a dividir los datos originales en dos, un conjunto de entrenamiento y otro de test, y procedamos de nuevo.

In [ ]:
# Creamos el conjunto de entrenamiento
# tomando aleatoriamente el 80% de filas
body_train = body.copy().sample(frac=0.8)
# El conjunto de test es el formado
# por el resto de filas
body_test = body.copy().drop(body_train.index)

# Variable respuesta
y_train = body_train['Gender']
y_test = body_test['Gender']
# Las variables predictoras son todas las demás
X_train = body_train.iloc[:,1:]
X_test = body_test.iloc[:,1:]
# Añadimos el término constante
X_train = sm.add_constant(X_train, prepend=False)
X_test = sm.add_constant(X_test, prepend=False)

# Construimos el modelo logístico
model = smf.Logit(y_train, X_train)
# Y ajustamos
mfitted = model.fit()


Optimization terminated successfully.
         Current function value: 0.135222
         Iterations 9


Ahora realizamos la predicción sobre el conjunto de test.

In [ ]:
pred = mfitted.predict(X_test)
pred.head()

1     0.983750
4     0.999973
8     0.986980
14    0.993570
27    0.998867
dtype: float64

Los valores de la predicción para cada individuo están en el rango [0, 1]. Interpretando los resultados como _Prob(y = 1)_, tomamos que valores (p >= 0.5) corresponden a la categoría 1. Calculemos la tabla de confusión.

In [ ]:
pred_0_1 = [ 1 if p >= 0.5 else 0 for p in pred ]

pd.crosstab(index=y_test, columns=pd.Categorical(pred_0_1), rownames=['Obs'], colnames=['Pred'])

Pred,0,1
Obs,,
0,46,6
1,2,47


Y por último, calculemos el acierto global.

In [ ]:
acierto_total = np.mean(y_test == pred_0_1)
print ('acierto_total = {0}%'.format(np.round(acierto_total*100, 2))  )


acierto_total = 92.08%
